In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from dotenv import load_dotenv
load_dotenv("D:/work4/activeloop_LLM/.env")

In [ ]:
from langchain.llms import GooglePalm
google_api_key = os.getenv("GOOGLE_PALM_API_KEY")

In [ ]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
llm = GooglePalm(google_api_key=google_api_key, temperature=0.7)

In [ ]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities from Monday to Sunday with each day in a sentence."
print(llm(text))

In [ ]:
#from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
#download_location = "model/"
#instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", \
#                                                      cache_folder=download_location)
#e = instructor_embeddings.embed_query("What is your refund policy?")
#print(len(e))
#print(e[:5])

#### The Chains

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))

#### The Memory

In [ ]:
from langchain.llms import GooglePalm
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = GooglePalm(google_api_key=google_api_key, temperature=0.0)

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

#Start the conversation
conversation.predict(input="Tell me about yourself.")

#Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)


In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/text-embedding-ada-002', cache_dir="model/gpt2")

In [ ]:
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
# instantiate the LLM and embeddings models
llm = GooglePalm(google_api_key=google_api_key, temperature=0)
embedder = tokenizer

In [ ]:
# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [ ]:
type(docs)

In [ ]:
for doc in docs:
    print(doc)

In [ ]:
embedded_docs = []
for doc in docs:
    embedded_docs.append(embedder.encode(doc.page_content))

In [ ]:
for doc in embedded_docs:
    print(doc)

In [ ]:
# create Deep Lake dataset
# org id is your username
my_activeloop_org_id = os.getenv("ACTIVELOOP_ORG_ID") 
my_activeloop_dataset_name = "course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=None)

In [ ]:
# add documents to our Deep Lake dataset
db.add_documents(embedded_docs)